##🤖 Eastern Services RAG Chatbot - Complete System

**Features:**
- ✅ Multilingual Support (English, Urdu, Roman Urdu, Punjabi)
- ✅ RAG with FAISS Vector Store
- ✅ Conversation Memory (Redis + Supabase)
- ✅ Fallback Mechanism
- ✅ Metadata Filtering
- ✅ Evaluation Metrics
- ✅ Secure .env Configuration

**Database Setup:**
- **PostgreSQL**: Supabase (User data, conversation history)
- **Redis**: Upstash (Session cache, fast memory)
- **FAISS**: Local vector store (Document embeddings)
```

Step 1:**Installation and Setup**

Installs all necessary dependencies (langchain, faiss-cpu, sentence-transformers, huggingface-hub) and mounts Google Drive. We rely on the core packages being installed cleanly now.

In [1]:
# ============================================================================
# CELL 1: Installation and Setup (NO GOOGLE DRIVE)
# ============================================================================

print("="*80)
print("INSTALLING DEPENDENCIES")
print("="*80)

# Install all required packages
!pip install -q \
langchain-groq \
langchain \
langchain-community \
langchain-core \
langchain-text-splitters \
sentence-transformers \
huggingface-hub \
python-docx \
python-dotenv \
redis \
psycopg2-binary \
sqlalchemy \
cohere \
requests \
PyPDF2 \
python-pptx \
openpyxl \
fastapi \
uvicorn

print("✅ All packages installed!")
print("="*80)
print("\n✅ Dependencies installed successfully.")
print("✅ Supported formats: DOCX, PDF, PPTX, XLSX")
print("✅ Data source: GitHub repository")
print("✅ Embeddings: Supabase PGVector")
print("✅ Environment: Hugging Face Spaces")



INSTALLING DEPENDENCIES
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

## Cell 2 - Load Environment Variables

In [4]:
# Cell 2: Load Environment Variables (Works in BOTH Colab AND Hugging Face Spaces)

import os
from dotenv import load_dotenv

print("\n" + "="*80)
print("LOADING ENVIRONMENT VARIABLES")
print("="*80)

# ============================================================================
# STRATEGY: Try multiple sources (Colab secrets, .env file, HF Spaces secrets)
# ============================================================================

GROQ_API_KEY = None
COHERE_API_KEY = None
SUPABASE_URI = None
REDIS_URI = None

# Method 1: Try Google Colab Secrets (if running in Colab)
try:
    from google.colab import userdata
    print("📱 Detected: Google Colab environment")
    print("   Trying to load from Colab secrets...")

    try:
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
        COHERE_API_KEY = userdata.get('COHERE_API_KEY')
        SUPABASE_URI = userdata.get('SUPABASE_URI')
        REDIS_URI = userdata.get('REDIS_URI')
        print("   ✅ Loaded from Colab secrets!")
    except Exception as e:
        print(f"   ⚠️  Colab secrets not found: {e}")
        print("   💡 To use Colab secrets:")
        print("      Go to: 🔑 (key icon) → Secrets → Add secret")
        print("      Add: GROQ_API_KEY, COHERE_API_KEY, SUPABASE_URI, REDIS_URI")
except ImportError:
    print("📱 Not running in Google Colab")

# Method 2: Try .env file (if not loaded from Colab secrets)
if not all([GROQ_API_KEY, COHERE_API_KEY, SUPABASE_URI, REDIS_URI]):
    print("\n📄 Trying to load from .env file...")

    # Try multiple possible .env file locations
    env_paths = [
        '/content/.env',  # Colab root
        '/content/drive/MyDrive/Eastern_chatbot/.env',  # Google Drive
        '.env',  # Current directory
        os.path.expanduser('~/.env'),  # Home directory
    ]

    env_loaded = False
    for env_path in env_paths:
        if os.path.exists(env_path):
            load_dotenv(env_path)
            print(f"   ✅ Found .env file at: {env_path}")
            env_loaded = True
            break

    if not env_loaded:
        print("   ⚠️  No .env file found")
        print("   💡 To use .env file:")
        print("      Create a .env file with:")
        print("      GROQ_API_KEY=your_key")
        print("      COHERE_API_KEY=your_key")
        print("      SUPABASE_URI=your_uri")
        print("      REDIS_URI=your_uri")

# Method 3: Try environment variables (works in HF Spaces and local)
if not all([GROQ_API_KEY, COHERE_API_KEY, SUPABASE_URI, REDIS_URI]):
    print("\n🌐 Trying to load from environment variables...")
    GROQ_API_KEY = os.getenv('GROQ_API_KEY') or GROQ_API_KEY
    COHERE_API_KEY = os.getenv('COHERE_API_KEY') or COHERE_API_KEY
    SUPABASE_URI = os.getenv('SUPABASE_URI') or SUPABASE_URI
    REDIS_URI = os.getenv('REDIS_URI') or REDIS_URI

    if all([GROQ_API_KEY, COHERE_API_KEY, SUPABASE_URI, REDIS_URI]):
        print("   ✅ Loaded from environment variables!")
        print("   📍 This works in Hugging Face Spaces automatically")

# Final check: Load from os.getenv if still missing (after load_dotenv)
if not GROQ_API_KEY:
    GROQ_API_KEY = os.getenv('GROQ_API_KEY')
if not COHERE_API_KEY:
    COHERE_API_KEY = os.getenv('COHERE_API_KEY')
if not SUPABASE_URI:
    SUPABASE_URI = os.getenv('SUPABASE_URI')
if not REDIS_URI:
    REDIS_URI = os.getenv('REDIS_URI')

# ============================================================================
# VERIFY ALL VARIABLES ARE LOADED
# ============================================================================

required_vars = {
    'GROQ_API_KEY': GROQ_API_KEY,
    'COHERE_API_KEY': COHERE_API_KEY,
    'SUPABASE_URI': SUPABASE_URI,
    'REDIS_URI': REDIS_URI
}

missing_vars = [k for k, v in required_vars.items() if not v]

if missing_vars:
    print("\n" + "="*80)
    print("❌ ERROR: Missing required environment variables")
    print("="*80)
    print(f"Missing: {missing_vars}\n")

    print("📋 SOLUTIONS:")
    print("\n1️⃣  FOR GOOGLE COLAB (Recommended for testing):")
    print("   a) Click the 🔑 (key icon) in the left sidebar")
    print("   b) Click 'Secrets' tab")
    print("   c) Click 'Add secret' for each:")
    print("      - GROQ_API_KEY")
    print("      - COHERE_API_KEY")
    print("      - SUPABASE_URI")
    print("      - REDIS_URI")
    print("   d) Re-run this cell")

    print("\n2️⃣  FOR LOCAL/ENV FILE:")
    print("   Create a .env file with:")
    print("   GROQ_API_KEY=your_key_here")
    print("   COHERE_API_KEY=your_key_here")
    print("   SUPABASE_URI=your_uri_here")
    print("   REDIS_URI=your_uri_here")

    print("\n3️⃣  FOR HUGGING FACE SPACES:")
    print("   a) Go to your Space → Settings → Variables and secrets")
    print("   b) Click 'New secret' for each variable")
    print("   c) Add as 'Secret' (not 'Variable')")
    print("   d) Deploy")

    raise ValueError(f"Missing environment variables: {missing_vars}")

# ============================================================================
# SUCCESS
# ============================================================================

print("\n" + "="*80)
print("✅ ALL ENVIRONMENT VARIABLES LOADED SUCCESSFULLY!")
print("="*80)
print(f"   - GROQ_API_KEY: {'*' * 20} ✅")
print(f"   - COHERE_API_KEY: {'*' * 20} ✅")
print(f"   - SUPABASE_URI: {SUPABASE_URI[:30]}... ✅")
print(f"   - REDIS_URI: {REDIS_URI[:25]}... ✅")
print("="*80)




LOADING ENVIRONMENT VARIABLES
📱 Detected: Google Colab environment
   Trying to load from Colab secrets...
   ✅ Loaded from Colab secrets!

✅ ALL ENVIRONMENT VARIABLES LOADED SUCCESSFULLY!
   - GROQ_API_KEY: ******************** ✅
   - COHERE_API_KEY: ******************** ✅
   - SUPABASE_URI: postgresql://postgres.kcyrxqcv... ✅
   - REDIS_URI: rediss://default:ARyhAAIm... ✅


## Cell 3 - Configuration and Paths

In [6]:
# Cell 3: Configuration and Paths (FIXED - Uses same loading method as Cell 2)

import os
from dotenv import load_dotenv

# ============================================================================
# LOAD ENVIRONMENT VARIABLES (Same method as Cell 2)
# ============================================================================

GROQ_API_KEY = None
COHERE_API_KEY = None
SUPABASE_URI = None
REDIS_URI = None

# Method 1: Try Google Colab Secrets (if running in Colab)
try:
    from google.colab import userdata
    try:
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
        COHERE_API_KEY = userdata.get('COHERE_API_KEY')
        SUPABASE_URI = userdata.get('SUPABASE_URI')
        REDIS_URI = userdata.get('REDIS_URI')
    except Exception:
        pass
except ImportError:
    pass

# Method 2: Try .env file (if not loaded from Colab secrets)
if not all([GROQ_API_KEY, COHERE_API_KEY, SUPABASE_URI, REDIS_URI]):
    env_paths = [
        '/content/.env',
        '/content/drive/MyDrive/Eastern_chatbot/.env',
        '.env',
        os.path.expanduser('~/.env'),
    ]

    for env_path in env_paths:
        if os.path.exists(env_path):
            load_dotenv(env_path)
            break

# Method 3: Try environment variables (works in HF Spaces and local)
if not GROQ_API_KEY:
    GROQ_API_KEY = os.getenv('GROQ_API_KEY')
if not COHERE_API_KEY:
    COHERE_API_KEY = os.getenv('COHERE_API_KEY')
if not SUPABASE_URI:
    SUPABASE_URI = os.getenv('SUPABASE_URI')
if not REDIS_URI:
    REDIS_URI = os.getenv('REDIS_URI')

# ============================================================================
# CONFIGURATION
# ============================================================================

GITHUB_REPO_URL = "https://raw.githubusercontent.com/abdullahnaseem4802-dot/Eastern-Chatbot-Data/main/"

DATA_FILES = [
    "Ant_Control_Services.docx",
    "Bed_Bug_Treatment_Guide.docx",
    "Booking_and_Scheduling.docx",
    "Chemical_Safety_Information.docx",
    "Cockroach_Control_Services.docx",
    "Commercial_Pest_Control.docx",
    "Comprehensive_FAQ.docx",
    "Contact_Support_Information.docx",
    "Corporate_Packages_2025.docx",
    "Customer_Testimonials.docx",
    "Detailed_Pricing_2025.docx",
    "Emergency_Pest_Services.docx",
    "Fumigation_Services.docx",
    "Mosquito_Control_Services.docx",
    "Pre_Construction_Treatment.docx",
    "Rodent_Control_Solutions.docx",
    "Seasonal_Pest_Guide.docx",
    "Service_Areas_Coverage.docx",
    "Seasonal_Pest_Calendar.xlsx",
    "Pest_Identification_Chart.pptx",
    "Monthly_Service_Plans.xlsx",
    "Integrated_Pest_Management.pdf",
    "Installation_Methods_Guide.pdf",
    "DIY_Prevention_Tips.pptx",
    "Termite_Treatment_Complete.docx",
    "Warranty_Guarantee_Policy.docx"
]

# ============================================================================
# VERIFY ALL KEYS ARE LOADED
# ============================================================================

required_keys = {
    'GROQ_API_KEY': GROQ_API_KEY,
    'COHERE_API_KEY': COHERE_API_KEY,
    'SUPABASE_URI': SUPABASE_URI,
    'REDIS_URI': REDIS_URI
}

missing_keys = [k for k, v in required_keys.items() if not v]

if missing_keys:
    print("❌ ERROR: Missing required environment variables!")
    print(f"Missing: {missing_keys}\n")
    print("💡 SOLUTION:")
    print("   1. Make sure Cell 2 ran successfully")
    print("   2. If using Colab, add secrets in 🔑 → Secrets")
    print("   3. Re-run Cell 2, then Cell 3")
    raise ValueError(f"Missing: {missing_keys}")

print("✅ Configuration loaded!")
print(f"   Files: {len(DATA_FILES)}")
print(f"   Source: GitHub (Public)")
print(f"   ✅ All API keys loaded successfully!")
print("="*80)



✅ Configuration loaded!
   Files: 26
   Source: GitHub (Public)
   ✅ All API keys loaded successfully!


## Cell 4 - Database Connections (Redis + Supabase with PGVector)

In [7]:
# Cell 4: Initialize Database Connections (Redis + Supabase with PGVector)

import redis
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, text
from sqlalchemy.orm import declarative_base, sessionmaker
from datetime import datetime
import json

print("="*80)
print("CONNECTING TO DATABASES")
print("="*80)

# --- REDIS CONNECTION ---
REDIS_URI_CLEAN = REDIS_URI.replace('REDIS_URI=', '').strip()

print("\n📡 Connecting to Redis (Upstash)...")
try:
    redis_client = redis.from_url(
        REDIS_URI_CLEAN,
        decode_responses=True,
        socket_connect_timeout=5
    )
    redis_client.ping()
    print("✅ Redis connected successfully!")
except Exception as e:
    print(f"❌ Redis connection failed: {e}")
    redis_client = None

# --- SUPABASE (POSTGRESQL) CONNECTION WITH PGVECTOR ---
print("\n📡 Connecting to Supabase (PostgreSQL + PGVector)...")

SessionLocal = None
ConversationHistory = None
engine = None

try:
    # Create engine with robust settings
    engine = create_engine(
        SUPABASE_URI,
        pool_pre_ping=True,
        pool_recycle=3600,
        pool_size=5,
        max_overflow=10,
        connect_args={
            'connect_timeout': 15,
            'application_name': 'eastern_chatbot_production'
        }
    )

    # Test connection
    print("🔍 Testing database connection...")
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"✅ Connection successful!")
        print(f"   PostgreSQL: {version[:60]}...")

        # Check if pgvector extension is enabled
        result = conn.execute(text("SELECT * FROM pg_extension WHERE extname = 'vector';"))
        if result.fetchone():
            print("✅ PGVector extension is enabled!")
        else:
            print("⚠️  PGVector extension not found - enabling it now...")
            conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
            conn.commit()
            print("✅ PGVector extension enabled!")

    # Define schema
    Base = declarative_base()

    class ConversationHistory(Base):
        __tablename__ = 'conversation_history'

        id = Column(Integer, primary_key=True, autoincrement=True)
        session_id = Column(String(100), index=True)
        user_question = Column(Text)
        bot_response = Column(Text)
        language = Column(String(50))
        timestamp = Column(DateTime, default=datetime.utcnow)
        meta_data = Column(Text)

    # Create tables
    print("📋 Creating/verifying database tables...")
    Base.metadata.create_all(engine)

    SessionLocal = sessionmaker(bind=engine)

    print("✅ Supabase connected successfully!")
    print("✅ Database tables created/verified!")
    print("✅ PGVector ready for embeddings!")

except Exception as e:
    error_msg = str(e)
    print(f"❌ Supabase connection failed!")
    print(f"   Error: {error_msg[:100]}...")
    print("\n🔧 TROUBLESHOOTING:")
    print("   1. Check if Supabase project is active")
    print("   2. Verify password in .env file")
    print("   3. Ensure PGVector extension is enabled in Supabase dashboard")
    SessionLocal = None
    engine = None

# --- SUMMARY ---
print("\n" + "="*80)
print("DATABASE CONNECTION STATUS")
print("="*80)
print(f"{'✅' if redis_client else '❌'} Redis (Upstash): {'Connected' if redis_client else 'Failed'}")
print(f"{'✅' if SessionLocal else '❌'} Supabase (PostgreSQL): {'Connected' if SessionLocal else 'Failed'}")
print(f"{'✅' if engine else '❌'} PGVector: {'Ready' if engine else 'Not Available'}")

if redis_client and SessionLocal and engine:
    print("\n🎉 FULL CLOUD SYSTEM OPERATIONAL!")
    print("   ✅ Redis: Session cache")
    print("   ✅ Supabase: Permanent storage + PGVector embeddings")
    print("   ✅ Data Source: GitHub")
elif SessionLocal and engine:
    print("\n⚠️  PARTIAL SYSTEM OPERATIONAL")
    print("   ❌ Redis: Not available")
    print("   ✅ Supabase: Connected with PGVector")
else:
    print("\n❌ SYSTEM NOT OPERATIONAL")
    print("   Fix database connections before proceeding")

print("="*80)

CONNECTING TO DATABASES

📡 Connecting to Redis (Upstash)...
✅ Redis connected successfully!

📡 Connecting to Supabase (PostgreSQL + PGVector)...
🔍 Testing database connection...
✅ Connection successful!
   PostgreSQL: PostgreSQL 17.6 on aarch64-unknown-linux-gnu, compiled by gc...
✅ PGVector extension is enabled!
📋 Creating/verifying database tables...
✅ Supabase connected successfully!
✅ Database tables created/verified!
✅ PGVector ready for embeddings!

DATABASE CONNECTION STATUS
✅ Redis (Upstash): Connected
✅ Supabase (PostgreSQL): Connected
✅ PGVector: Ready

🎉 FULL CLOUD SYSTEM OPERATIONAL!
   ✅ Redis: Session cache
   ✅ Supabase: Permanent storage + PGVector embeddings
   ✅ Data Source: GitHub


## Cell 5 - Document Loading and Chunking

In [8]:
# Cell 5: Document Loading from GitHub (Multi-Format Support)

import requests
from io import BytesIO
import json

def download_file_from_github(url, filename):
    """Download a file from GitHub."""
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        return response.content
    except Exception as e:
        print(f"   ⚠️  Failed to download {filename}: {e}")
        return None

def extract_text_from_docx(file_content):
    """Extract text from .docx file."""
    try:
        import docx
        doc = docx.Document(BytesIO(file_content))
        full_text = []
        for para in doc.paragraphs:
            if para.text.strip():
                full_text.append(para.text)

        # Also extract text from tables
        for table in doc.tables:
            for row in table.rows:
                row_text = ' | '.join([cell.text.strip() for cell in row.cells if cell.text.strip()])
                if row_text:
                    full_text.append(row_text)

        return '\n\n'.join(full_text)
    except Exception as e:
        print(f"   ❌ Error extracting DOCX: {e}")
        return None

def extract_text_from_pdf(file_content):
    """Extract text from PDF file."""
    try:
        import PyPDF2
        pdf_file = BytesIO(file_content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        full_text = []
        for page_num, page in enumerate(pdf_reader.pages):
            text = page.extract_text()
            if text.strip():
                full_text.append(text)

        return '\n\n'.join(full_text)
    except Exception as e:
        print(f"   ❌ Error extracting PDF: {e}")
        return None

def extract_text_from_pptx(file_content):
    """Extract text from PowerPoint file."""
    try:
        from pptx import Presentation
        pptx_file = BytesIO(file_content)
        prs = Presentation(pptx_file)

        full_text = []
        for slide_num, slide in enumerate(prs.slides, 1):
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    slide_text.append(shape.text)

            if slide_text:
                full_text.append(f"Slide {slide_num}:\n" + '\n'.join(slide_text))

        return '\n\n'.join(full_text)
    except Exception as e:
        print(f"   ❌ Error extracting PPTX: {e}")
        return None

def extract_text_from_excel(file_content):
    """Extract text from Excel file."""
    try:
        import openpyxl
        excel_file = BytesIO(file_content)
        workbook = openpyxl.load_workbook(excel_file, data_only=True)

        full_text = []
        for sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]
            sheet_text = [f"Sheet: {sheet_name}"]

            for row in sheet.iter_rows(values_only=True):
                row_text = ' | '.join([str(cell) for cell in row if cell is not None])
                if row_text.strip():
                    sheet_text.append(row_text)

            if len(sheet_text) > 1:  # More than just the sheet name
                full_text.append('\n'.join(sheet_text))

        return '\n\n'.join(full_text)
    except Exception as e:
        print(f"   ❌ Error extracting Excel: {e}")
        return None

def extract_text_from_file(file_content, filename):
    """Extract text based on file extension."""
    extension = filename.lower().split('.')[-1]

    extractors = {
        'docx': extract_text_from_docx,
        'pdf': extract_text_from_pdf,
        'pptx': extract_text_from_pptx,
        'xlsx': extract_text_from_excel,
        'xls': extract_text_from_excel
    }

    extractor = extractors.get(extension)
    if extractor:
        return extractor(file_content)
    else:
        print(f"   ⚠️  Unsupported file type: .{extension}")
        return None

def simple_chunk_text(text, chunk_size=1000, overlap=200):
    """Simple text chunking."""
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]

        if end < text_length:
            for separator in ['\n\n', '\n', '. ', ' ']:
                last_sep = chunk.rfind(separator)
                if last_sep > chunk_size * 0.5:
                    chunk = text[start:start + last_sep + len(separator)]
                    break

        chunks.append(chunk.strip())
        start = end - overlap

    return chunks

def load_documents_from_github(repo_url, file_list):
    """Load all documents from GitHub repository."""
    all_documents = []
    print(f"Loading documents from GitHub repository...")
    print(f"Repository: {repo_url}")
    print(f"Total files to load: {len(file_list)}\n")

    for filename in file_list:
        file_url = repo_url + filename
        print(f"📥 Downloading: {filename}")

        file_content = download_file_from_github(file_url, filename)

        if file_content:
            try:
                text = extract_text_from_file(file_content, filename)

                if text:
                    all_documents.append({
                        'content': text,
                        'source': filename,
                        'file_type': filename.split('.')[-1]
                    })
                    print(f"   ✅ Loaded: {filename} ({len(text)} characters)")
                else:
                    print(f"   ⚠️  No text extracted from {filename}")

            except Exception as e:
                print(f"   ❌ Error loading {filename}: {e}")
        else:
            print(f"   ❌ Skipped: {filename}")

    if not all_documents:
        raise FileNotFoundError(f"No documents loaded from GitHub!")

    print(f"\n✅ Successfully loaded {len(all_documents)} documents from GitHub")
    return all_documents

def create_chunks_from_documents(documents):
    """Create chunks from all documents."""
    print("\nSplitting documents into chunks...")
    all_chunks = []

    for doc in documents:
        text = doc['content']
        source = doc['source']

        text_chunks = simple_chunk_text(text, chunk_size=1000, overlap=200)

        for i, chunk_text in enumerate(text_chunks):
            chunk = {
                'text': chunk_text,
                'metadata': {
                    'source': source,
                    'file_type': doc['file_type'],
                    'chunk_index': i,
                    'total_chunks': len(text_chunks)
                }
            }
            all_chunks.append(chunk)

    print(f"✅ Created {len(all_chunks)} chunks")
    return all_chunks

# Execute
print("="*80)
print("LOADING DATA FROM GITHUB")
print("="*80)

try:
    documents = load_documents_from_github(GITHUB_REPO_URL, DATA_FILES)
    chunks = create_chunks_from_documents(documents)

    print("\n✅ Data preparation complete!")
    print(f"   Source: GitHub Repository")
    print(f"   Total Documents: {len(documents)}")
    print(f"   Total Chunks: {len(chunks)}")

    LOADED_CHUNKS = chunks

except Exception as e:
    print(f"❌ ERROR in data loading: {e}")
    import traceback
    traceback.print_exc()
    raise

print("="*80)

LOADING DATA FROM GITHUB
Loading documents from GitHub repository...
Repository: https://raw.githubusercontent.com/abdullahnaseem4802-dot/Eastern-Chatbot-Data/main/
Total files to load: 26

📥 Downloading: Ant_Control_Services.docx
   ✅ Loaded: Ant_Control_Services.docx (728 characters)
📥 Downloading: Bed_Bug_Treatment_Guide.docx
   ✅ Loaded: Bed_Bug_Treatment_Guide.docx (1128 characters)
📥 Downloading: Booking_and_Scheduling.docx
   ✅ Loaded: Booking_and_Scheduling.docx (963 characters)
📥 Downloading: Chemical_Safety_Information.docx
   ✅ Loaded: Chemical_Safety_Information.docx (2849 characters)
📥 Downloading: Cockroach_Control_Services.docx
   ✅ Loaded: Cockroach_Control_Services.docx (1575 characters)
📥 Downloading: Commercial_Pest_Control.docx
   ✅ Loaded: Commercial_Pest_Control.docx (2678 characters)
📥 Downloading: Comprehensive_FAQ.docx
   ✅ Loaded: Comprehensive_FAQ.docx (3542 characters)
📥 Downloading: Contact_Support_Information.docx
   ✅ Loaded: Contact_Support_Information.d

## Cell 6 - Create/Load FAISS Index

In [9]:
# Cell 6: COMPREHENSIVE URDU TRANSLITERATION + KEYWORD RE-RANKING

import cohere
from sqlalchemy import text
import json
import re

print("="*80)
print("PGVECTOR EMBEDDINGS SETUP")
print("="*80)

# Initialize Cohere client
print("\n🔌 Initializing Cohere embeddings...")
try:
    cohere_client = cohere.Client(COHERE_API_KEY)
    print("✅ Cohere client initialized!")
except Exception as e:
    print(f"❌ Cohere initialization failed: {e}")
    raise

# Create embeddings table in Supabase if it doesn't exist
print("\n📋 Setting up embeddings table in Supabase...")

try:
    with engine.connect() as conn:
        # Create embeddings table with vector column
        conn.execute(text("""
            CREATE TABLE IF NOT EXISTS document_embeddings (
                id SERIAL PRIMARY KEY,
                chunk_text TEXT NOT NULL,
                embedding vector(1024),
                metadata JSONB,
                created_at TIMESTAMP DEFAULT NOW()
            );
        """))

        # Create index for faster similarity search
        conn.execute(text("""
            CREATE INDEX IF NOT EXISTS embedding_idx
            ON document_embeddings
            USING ivfflat (embedding vector_cosine_ops)
            WITH (lists = 100);
        """))

        conn.commit()
        print("✅ Embeddings table created/verified!")

        # Check if embeddings already exist
        result = conn.execute(text("SELECT COUNT(*) FROM document_embeddings;"))
        existing_count = result.fetchone()[0]

        if existing_count > 0:
            print(f"✅ Found {existing_count} existing embeddings in database")
            print("   Skipping re-embedding (data already indexed)")
            embeddings_exist = True
        else:
            print("   No existing embeddings found")
            embeddings_exist = False

except Exception as e:
    print(f"❌ Error setting up embeddings table: {e}")
    raise

# Generate and store embeddings if they don't exist
if not embeddings_exist:
    print("\n🔄 Generating embeddings for all chunks...")
    print(f"   Total chunks to embed: {len(LOADED_CHUNKS)}")
    print("   This may take a few minutes...")

    try:
        # Process in batches (Cohere API limit: 96 texts per call)
        batch_size = 90
        total_embedded = 0

        for i in range(0, len(LOADED_CHUNKS), batch_size):
            batch = LOADED_CHUNKS[i:i + batch_size]
            batch_texts = [chunk['text'] for chunk in batch]

            print(f"   Processing batch {i//batch_size + 1}/{(len(LOADED_CHUNKS)-1)//batch_size + 1}...")

            # Get embeddings from Cohere
            response = cohere_client.embed(
                texts=batch_texts,
                model='embed-english-v3.0',
                input_type='search_document'
            )

            embeddings_batch = response.embeddings

            # Insert into Supabase - USE RAW CONNECTION
            raw_conn = engine.raw_connection()
            try:
                cursor = raw_conn.cursor()

                for chunk, embedding in zip(batch, embeddings_batch):
                    # Convert embedding to PostgreSQL vector format
                    embedding_str = '[' + ','.join(map(str, embedding)) + ']'
                    metadata_json = json.dumps(chunk['metadata'])

                    cursor.execute("""
                        INSERT INTO document_embeddings (chunk_text, embedding, metadata)
                        VALUES (%s, %s::vector, %s::jsonb)
                    """, (chunk['text'], embedding_str, metadata_json))

                    total_embedded += 1

                raw_conn.commit()
                cursor.close()
            finally:
                raw_conn.close()

            print(f"   ✅ Embedded and stored {total_embedded}/{len(LOADED_CHUNKS)} chunks")

        print(f"\n✅ All embeddings generated and stored in Supabase!")
        print(f"   Total chunks embedded: {total_embedded}")

    except Exception as e:
        print(f"❌ Error generating embeddings: {e}")
        import traceback
        traceback.print_exc()
        raise

# COMPREHENSIVE Urdu transliteration helper
def transliterate_urdu_to_roman(text):
    """Comprehensive Urdu to Roman transliteration for better embedding matching."""
    urdu_pattern = r'[\u0600-\u06FF]'
    if not re.search(urdu_pattern, text):
        return text

    # EXPANDED: 200+ word mapping
    urdu_to_roman = {
        # Core pest terms
        'دیمک': 'deemak termite white ant wood damage',
        'کھٹمل': 'khatmal bed bug bedbug mattress',
        'کاکروچ': 'cockroach roach kitchen pest',
        'مچھر': 'mosquito machar dengue malaria',
        'چوہے': 'rat rodent mouse chooha chuhay',
        'چوہا': 'rat rodent mouse chooha',
        'چیونٹی': 'ant chiti sugar ant',

        # Service terms
        'علاج': 'ilaj treatment control service',
        'سروس': 'service treatment ilaj',
        'خدمات': 'services khidmat',
        'کنٹرول': 'control treatment',
        'سپرے': 'spray chemical treatment',
        'فیومیگیشن': 'fumigation',
        'معائنہ': 'inspection check survey',

        # Pricing terms
        'ریٹ': 'rate price cost qeemat',
        'قیمت': 'qeemat price rate cost',
        'رقم': 'amount money price',
        'روپے': 'rupees rs money',
        'فی': 'per',
        'مربع': 'square sq',
        'فٹ': 'foot ft feet',
        'مرلہ': 'marla',
        'کنال': 'kanal',

        # Guarantee terms
        'گارنٹی': 'guarantee warranty assurance',
        'ضمانت': 'guarantee warranty',
        'وارنٹی': 'warranty guarantee',

        # Time terms
        'کتنی': 'kitni how much how many',
        'کتنا': 'kitna how much',
        'کب': 'kab when',
        'کیسے': 'kaisay how',
        'کیوں': 'kyun why',
        'وقت': 'waqt time duration',
        'دن': 'day din',
        'ماہ': 'month mahina',
        'سال': 'year saal',

        # Location terms
        'گھر': 'ghar home house residence',
        'مکان': 'house home makan',
        'علاقہ': 'area ilaqa region',
        'علاقے': 'areas ilaqay regions',
        'شہر': 'city shahar',
        'لکڑی': 'wood lakri timber',
        'تعمیر': 'construction building tameer',

        # Question words
        'کیا': 'kya kia what',
        'کون': 'kon kaun which who what',
        'کہاں': 'kahan where',
        'کس': 'kis which',
        'کونسا': 'konsa which',
        'کونسی': 'konsi which',
        'کونسے': 'konsay which',

        # Common verbs
        'ہے': 'hai is',
        'ہیں': 'hain are',
        'ہو': 'ho',
        'گا': 'ga will',
        'گی': 'gi will',
        'گے': 'gay will',
        'ہوگا': 'hoga will be',
        'ہوگی': 'hogi will be',
        'ہوگے': 'hogay will be',
        'کریں': 'karein do',
        'کرنا': 'karna to do',
        'دیں': 'dein give',
        'دینا': 'dena to give',
        'آئیں': 'aayein come',
        'آنا': 'aana to come',
        'جائیں': 'jayein go',
        'جانا': 'jana to go',

        # Possessives
        'میرے': 'meray mery my',
        'میری': 'meri my',
        'میرا': 'mera my',
        'آپ': 'aap you',
        'آپکا': 'aapka your',
        'آپکی': 'aapki your',
        'آپکے': 'aapkay your',
        'ہمارے': 'humare our',
        'ہماری': 'humari our',
        'ہمارا': 'humara our',

        # Prepositions
        'میں': 'mein in',
        'کے': 'ke kay of',
        'کی': 'ki of',
        'کا': 'ka of',
        'سے': 'se say from with',
        'کو': 'ko to',
        'نے': 'ne',
        'پر': 'par on',
        'لیے': 'liye liyay for',
        'بارے': 'baray about',
        'ساتھ': 'saath with',

        # Adjectives
        'بہت': 'bohat bahut very much',
        'زیادہ': 'zyada more',
        'کم': 'kam less',
        'بڑا': 'bara big large',
        'چھوٹا': 'chota small',
        'اچھا': 'acha good',
        'بہترین': 'behtareen best excellent',
        'پیشہ ورانہ': 'professional peshawrana',
        'معیاری': 'quality standard',

        # Common nouns
        'مسئلہ': 'masla problem issue',
        'حل': 'solution hal',
        'طریقہ': 'method tariqa',
        'پیکج': 'package',
        'بکنگ': 'booking',
        'شیڈولنگ': 'scheduling',
        'رابطہ': 'contact rabta',
        'مدد': 'help madad',
        'معلومات': 'information maloomat',
        'تفصیل': 'details tafseel',

        # Negation
        'نہیں': 'nahi no not',
        'نہ': 'na no not',

        # Conjunctions
        'اور': 'aur and',
        'یا': 'ya or',
        'لیکن': 'lekin but',
        'مگر': 'magar but',
        'تو': 'to then',
        'اگر': 'agar if',

        # Polite terms
        'براہ': 'barah please',
        'کرم': 'karam kindness',
        'شکریہ': 'shukriya thanks',
        'معذرت': 'maazrat sorry',
        'خوش': 'khush happy',
        'آمدید': 'aamdeed welcome',

        # Business terms
        'کمپنی': 'company',
        'فرم': 'firm',
        'ٹیم': 'team',
        'ماہر': 'expert mahir',
        'تجربہ': 'experience tajurba',
        'ضرورت': 'need zaroorat',
        'فوری': 'urgent fori',
        'ایمرجنسی': 'emergency',

        # Additional context
        'پہلے': 'pehlay before first',
        'بعد': 'baad after',
        'دوران': 'during doran',
        'ختم': 'khatam end finish',
        'شروع': 'start shuru',
        'مکمل': 'complete mukammal',
    }

    # Replace Urdu words with Roman equivalents
    roman_text = text
    for urdu_word, roman_word in urdu_to_roman.items():
        roman_text = roman_text.replace(urdu_word, roman_word)

    # Remove any remaining non-Roman/non-space characters
    roman_text = re.sub(r'[^\w\s]', ' ', roman_text)
    roman_text = re.sub(r'\s+', ' ', roman_text)

    return roman_text.strip()

# Create retriever function for PGVector with KEYWORD RE-RANKING
def retrieve_similar_chunks(query, k=3):
    """Retrieve similar chunks from PGVector with Urdu support and keyword re-ranking."""
    try:
        # Transliterate Urdu to Roman for better matching
        search_query = transliterate_urdu_to_roman(query)
        if search_query != query:
            print(f"   🔄 Transliterated: '{query[:40]}...' → '{search_query[:40]}...'")

        # Generate query embedding
        query_response = cohere_client.embed(
            texts=[search_query],
            model='embed-english-v3.0',
            input_type='search_query'
        )
        query_embedding = query_response.embeddings[0]
        query_embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'

        # RETRIEVE MORE chunks for re-ranking (5 instead of 3)
        retrieval_k = k + 2

        # Search for similar chunks
        raw_conn = engine.raw_connection()
        try:
            cursor = raw_conn.cursor()

            cursor.execute("""
                SELECT chunk_text, metadata,
                       1 - (embedding <=> %s::vector) as similarity
                FROM document_embeddings
                ORDER BY embedding <=> %s::vector
                LIMIT %s
            """, (query_embedding_str, query_embedding_str, retrieval_k))

            rows = cursor.fetchall()
            cursor.close()

            documents = []
            for row in rows:
                metadata = row[1] if isinstance(row[1], dict) else (json.loads(row[1]) if row[1] else {})
                doc = {
                    'page_content': row[0],
                    'metadata': metadata,
                    'similarity': row[2]
                }
                documents.append(doc)

            # KEYWORD-BASED RE-RANKING
            query_lower = search_query.lower()

            # Define pest-specific keywords
            pest_keywords = {
                'deemak': ['termite', 'deemak', 'white ant', 'wood', 'termite treatment', 'post-construction', 'pre-construction'],
                'termite': ['termite', 'deemak', 'white ant', 'wood', 'termite treatment', 'post-construction', 'pre-construction'],
                'khatmal': ['bed bug', 'khatmal', 'bedbug', 'mattress', 'bed bug treatment'],
                'bed': ['bed bug', 'khatmal', 'bedbug', 'mattress', 'bed bug treatment'],
                'bug': ['bed bug', 'khatmal', 'bedbug', 'mattress', 'bed bug treatment'],
                'cockroach': ['cockroach', 'roach', 'kitchen', 'cockroach control'],
                'mosquito': ['mosquito', 'malaria', 'dengue', 'mosquito control', 'machar'],
                'rat': ['rat', 'rodent', 'mouse', 'chuhay', 'chooha', 'rodent control'],
                'chuhay': ['rat', 'rodent', 'mouse', 'chuhay', 'chooha', 'rodent control'],
                'ant': ['ant', 'chiti', 'sugar ant', 'ant control'],
                'guarantee': ['guarantee', 'warranty', 'guaranteed', 'assurance'],
                'rate': ['rate', 'price', 'cost', 'pricing', 'charges', 'rupees', 'rs', 'qeemat', 'per sq ft'],
                'service': ['service', 'services', 'provide', 'offer', 'ilaj', 'treatment', 'control'],
                'marla': ['marla', 'sq ft', 'square', 'area', 'size', 'construction'],
                'kanal': ['kanal', 'sq ft', 'square', 'area', 'size', 'construction'],
            }

            # Find which pest/topic is mentioned in query
            query_relevant_keywords = []
            for key, keywords in pest_keywords.items():
                if any(kw in query_lower for kw in [key]):
                    query_relevant_keywords.extend(keywords)

            # Ensure unique keywords
            query_relevant_keywords = list(set(query_relevant_keywords))

            # Re-rank documents based on keyword matches
            if query_relevant_keywords:
                for doc in documents:
                    doc_text_lower = doc['page_content'].lower()
                    keyword_matches = sum(1 for kw in query_relevant_keywords if kw in doc_text_lower)

                    # STRONG BOOST: Each keyword match adds 0.2
                    if keyword_matches > 0:
                        boost = min(0.5, keyword_matches * 0.2)
                        doc['similarity'] = min(1.0, doc['similarity'] + boost)
                        doc['keyword_matches'] = keyword_matches
                    else:
                        # PENALTY: Reduce similarity by 20% if NO keywords found
                        doc['similarity'] = doc['similarity'] * 0.8
                        doc['keyword_matches'] = 0

                # Re-sort by updated similarity
                documents.sort(key=lambda x: x['similarity'], reverse=True)

            # Return top k documents after re-ranking
            return documents[:k]

        finally:
            raw_conn.close()

    except Exception as e:
        print(f"❌ Error retrieving similar chunks: {e}")
        import traceback
        traceback.print_exc()
        return []

# Force functions into global scope
globals()['transliterate_urdu_to_roman'] = transliterate_urdu_to_roman
globals()['retrieve_similar_chunks'] = retrieve_similar_chunks

print("\n✅ PGVector retriever function created!")
print("   Use: retrieve_similar_chunks(query, k=3)")
print("   ⭐ NOW WITH IMPROVED URDU TRANSLITERATION!")
print("="*80)



PGVECTOR EMBEDDINGS SETUP

🔌 Initializing Cohere embeddings...
✅ Cohere client initialized!

📋 Setting up embeddings table in Supabase...
✅ Embeddings table created/verified!
✅ Found 67 existing embeddings in database
   Skipping re-embedding (data already indexed)

✅ PGVector retriever function created!
   Use: retrieve_similar_chunks(query, k=3)
   ⭐ NOW WITH IMPROVED URDU TRANSLITERATION!


## Cell 7 - Initialize LLM

In [10]:
from langchain_groq import ChatGroq

# Cell 7
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.3-70b-versatile",  # ← NEWEST MODEL!
    temperature=0.0
)

print("✅ Groq LLM initialized: Llama 3.3 70B Versatile (Latest)")
print("="*80)

✅ Groq LLM initialized: Llama 3.3 70B Versatile (Latest)


## Cell 8 - Language Detection and Helper Functions

In [11]:
# Cell 8: Language Detection (3 LANGUAGES ONLY - PUNJABI MAPPED TO URDU)

import re
from IPython.display import display, Markdown

def detect_greeting(text):
    """Detect if text contains a greeting."""
    text_lower = text.lower()
    urdu_pattern = r'[\u0600-\u06FF]'
    has_urdu_script = re.search(urdu_pattern, text)

    if 'السلام عليكم' in text or 'السلام' in text:
        return 'urdu_greeting'

    if 'assalam' in text_lower or 'assalamu' in text_lower:
        if has_urdu_script:
            return 'urdu_greeting'
        return 'roman_urdu_greeting'

    # Punjabi greeting mapped to Roman Urdu greeting
    if 'assalam' in text_lower or ('ki hall' in text_lower and 'ah pai' in text_lower):
        return 'roman_urdu_greeting'  # ← MAPPED TO ROMAN URDU

    if any(greet in text_lower for greet in ['hello', 'hi', 'hey', 'greetings']):
        return 'english_greeting'

    return None

def detect_language(text):
    """
    Detect language and MAP PUNJABI TO URDU.

    Returns ONLY 3 languages:
    - 'english' → English response
    - 'urdu' → Pure Urdu response
    - 'roman_urdu' → Roman Urdu response

    Punjabi queries are automatically mapped to appropriate Urdu.
    """
    text_lower = text.lower()

    # Check for Nastaliq/Urdu script (includes Punjabi Nastaliq)
    urdu_pattern = r'[\u0600-\u06FF]'
    if re.search(urdu_pattern, text):
        return 'urdu'  # ← ALL Nastaliq script = Pure Urdu response

    # Punjabi keywords (will be MAPPED to Roman Urdu)
    punjabi_keywords = [
        'tuhada', 'tussin', 'tusi', 'ah pai', 'ki hall', 'lag gi',
        'kaar cha', 'vich', 'da', 'di', 'oday', 'tay', 'karwana',
        'chana', 'karday', 'keri', 'oda', 'nu', 'te', 'ae',
        'samajh layya', 'dasso', 'layi', 'nal', 'baray', 'chahoge',
        'meray kaar', 'wood da kam', 'treatement da', 'ki ah'
    ]
    punjabi_count = sum(1 for keyword in punjabi_keywords if keyword in text_lower)

    # Roman Urdu keywords
    urdu_keywords = [
        'aapka', 'aapko', 'samajh gaya', 'hain', 'kia', 'hai',
        'karna', 'hoga', 'meray', 'ghar', 'deemak', 'rate',
        'ho ga', 'mein', 'ke', 'ka', 'ki', 'kitni', 'kitna',
        'chahta', 'chahiye', 'zaroorat', 'problem', 'masla',
        'ilaj', 'khatam', 'control', 'service', 'guarantee'
    ]
    urdu_count = sum(1 for keyword in urdu_keywords if keyword in text_lower)

    # English keywords
    english_keywords = [
        'what', 'which', 'how', 'when', 'where', 'why',
        'is', 'are', 'was', 'were', 'has', 'have', 'had',
        'the', 'this', 'that', 'these', 'those',
        'treatment', 'process', 'guarantee', 'period',
        'clients', 'worked', 'with', 'services', 'offer'
    ]
    english_count = sum(1 for keyword in english_keywords if keyword in text_lower)

    # 🎯 KEY CHANGE: If Punjabi detected, MAP TO ROMAN URDU
    if punjabi_count >= 2:
        print("  [🔄 Punjabi detected → Mapped to Roman Urdu]")
        return 'roman_urdu'  # ← PUNJABI → ROMAN URDU

    # If Roman Urdu keywords are dominant
    if urdu_count >= 3 and urdu_count > english_count:
        return 'roman_urdu'

    # If English keywords are dominant
    if english_count >= 3:
        return 'english'

    # Fallback checks
    if any(word in text_lower for word in ['kitni', 'kitna', 'kia', 'kya']) and english_count < 2:
        return 'roman_urdu'

    # 🎯 If Punjabi indicators found, map to Roman Urdu
    if any(word in text_lower for word in ['tusi', 'tuhada', 'karday', 'keri', 'vich', 'da di']):
        print("  [🔄 Punjabi detected → Mapped to Roman Urdu]")
        return 'roman_urdu'  # ← PUNJABI → ROMAN URDU

    # Default to English if unsure
    return 'english'

def clean_response(response, language, original_question=None):
    """Remove unwanted meta-commentary and clean up response."""
    patterns = [
        r'Q:\s*.*?\n\s*A:\s*',
        r'Based on the provided context.*?:\s*',
        r'According to the context.*?:\s*',
        r'RULE \d+.*?:\s*',
        r'INSTRUCTIONS?:.*?:\s*',
        r'Since the question.*?:\s*',
        r'The question is in.*?:\s*',
        r'LANGUAGE DETECTION.*?:\s*',
        r'I will.*?:\s*',
        r'Let me.*?:\s*',
        r'\([^)]*\)',
        r'Assalam-o-Alaikum!?\s*',
        r'السلام عليكم\s*',
    ]

    for pattern in patterns:
        response = re.sub(pattern, '', response, flags=re.IGNORECASE | re.DOTALL)

    # Remove repeated question if present
    if original_question:
        question_clean = original_question.strip()
        question_clean = re.sub(r'Assalam-o-Alaikum!?\s*', '', question_clean, flags=re.IGNORECASE)
        question_clean = re.sub(r'السلام عليكم\s*', '', question_clean)
        question_clean = re.sub(r'Assalam-o-Alaikum!?\s*', '', question_clean, flags=re.IGNORECASE)
        question_clean = question_clean.strip()

        response_stripped = response.strip()

        if question_clean and response_stripped.startswith(question_clean):
            temp_response = response_stripped[len(question_clean):].strip()
            if len(temp_response) > 20:
                response = temp_response

    response = response.strip()
    response = re.sub(r'\n\n+', '\n', response)
    response = re.sub(r'\s+', ' ', response)

    return response

print("✅ Language detection loaded!")
print("="*80)
print("SUPPORTED LANGUAGES (3 ONLY):")
print("  ✅ English → English response")
print("  ✅ Pure Urdu (Nastaliq) → Pure Urdu response")
print("  ✅ Roman Urdu → Roman Urdu response")
print("="*80)
print("LANGUAGE MAPPING:")
print("  🔄 Punjabi (Roman) → Roman Urdu response")
print("  🔄 Punjabi (Nastaliq) → Pure Urdu response")
print("="*80)



✅ Language detection loaded!
SUPPORTED LANGUAGES (3 ONLY):
  ✅ English → English response
  ✅ Pure Urdu (Nastaliq) → Pure Urdu response
  ✅ Roman Urdu → Roman Urdu response
LANGUAGE MAPPING:
  🔄 Punjabi (Roman) → Roman Urdu response
  🔄 Punjabi (Nastaliq) → Pure Urdu response


## Cell 9 - Language-Specific Prompts

In [12]:
# Cell 9: ULTIMATE FIX - Language-Specific Prompts (STOP RATE OBSESSION)

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ============================================================================
# 🚨 ULTRA-NUCLEAR PROMPTS - ANSWER ONLY WHAT IS ASKED
# ============================================================================

ENGLISH_PROMPT = """You are Eastern Services pest control bot.

Context: {context}

Question: {question}

⚠️ CRITICAL RULES - VIOLATION = IMMEDIATE FAIL:
1. Answer ONLY in English
2. Use ONLY Context information - NEVER make up information
3. Answer ONLY what the user asked - DO NOT add extra information
4. Keep answers 2-4 sentences
5. NO greetings unless user greets first
6. **CONTACT INFO:** Phone: +92 336 1101234, Email: easternservices.pk@gmail.com (ALWAYS use these, IGNORE any other phone numbers in Context!)
7. **GUARANTEE PERIODS:** Termite Treatment = 5 years, Termite Proofing = 10 years, General Pest Control = 1 month, Bed Bug Treatment = 3 months

**🚨 RATE/COST RULES - FOLLOW EXACTLY:**
- ❌ DO NOT mention rates/prices UNLESS the question explicitly asks about: rate, cost, price, charges, kitna, kitni
- ❌ DO NOT ask for area UNLESS user already mentioned a pest problem AND asked for cost calculation
- ✅ If asked "What services?", list services ONLY - NO rates
- ✅ If asked about process/method, explain process ONLY - NO rates
- ✅ If asked about guarantee, give guarantee period ONLY - NO rates

**CALCULATION RULES (only if user asks about cost/rate):**
- If user provides area (Marla, Kanal, sq ft), calculate total cost
- 1 Marla = 225 sq ft, 1 Kanal = 20 Marla = 4,500 sq ft
- Formula: Area (sq ft) × Rate per sq ft = Total Cost
- Example: 10 Marla = 2,250 sq ft × Rs. 80/sq ft = Rs. 180,000

**If user does NOT provide area and asks about rate:**
✅ CORRECT: "Termite treatment rate is Rs. 80 per sq ft."
❌ WRONG: "Please tell me your area so I can calculate..."

**EXAMPLES OF CORRECT RESPONSES:**
Q: "What services do you offer?"
A: "We offer Termite Treatment, Termite Proofing, General Pest Control, Bed Bug Treatment, Fumigation, Rodent Control, and Cockroach Control."

Q: "What is the termite treatment process?"
A: "We inspect the property, drill holes around the foundation, inject termiticide into the soil, treat all wooden structures, and provide a 5-year guarantee."

Q: "Do I need to vacate during treatment?"
A: "For fumigation, yes - you must vacate for 24-48 hours. For termite treatment, you can stay but avoid treated areas for 4-6 hours."

Answer (DIRECT, NO EXTRA INFO):"""

# ============================================================================

ROMAN_URDU_PROMPT = """Aap Eastern Services pest control bot hain.

Context: {context}

Question: {question}

⚠️ ZAROORI RULES - VIOLATION = FAIL:
1. Sirf Roman Urdu mein jawab do
2. Sirf Context ki info use karo - KABHI sawal wapas NA poocho
3. Sirf wahi jawab do jo user ne poocha - EXTRA info NA do
4. Jawab 2-4 sentences mein
5. NO salam jab tak user salam na karay
6. **CONTACT INFO:** Phone: +92 336 1101234, Email: easternservices.pk@gmail.com (HAMESHA yeh use karo!)
7. **GUARANTEE PERIODS:** Termite Treatment = 5 saal, Termite Proofing = 10 saal, General Pest Control = 1 mahina, Bed Bug Treatment = 3 mahine

**🚨 RATE/COST RULES - BILKUL FOLLOW KARO:**
- ❌ Rate/price KABHI NA batao JAB TAK sawal mein yeh na ho: rate, cost, price, charges, kitna, kitni
- ❌ Area KABHI NA poocho JAB TAK user ne pest problem + cost calculation dono NA bataya ho
- ✅ Agar "Kya services hain?" pucha, to SIRF services batao - NO rates
- ✅ Agar process/method pucha, to SIRF process batao - NO rates
- ✅ Agar guarantee pucha, to SIRF guarantee period batao - NO rates

**CALCULATION RULES (sirf agar user cost/rate puchay):**
- Agar user area bataye (Marla, Kanal, sq ft), to total cost CALCULATE karo
- 1 Marla = 225 sq ft, 1 Kanal = 20 Marla = 4,500 sq ft
- Formula: Area (sq ft) × Rate per sq ft = Total Cost
- Example: 10 Marla = 2,250 sq ft × Rs. 80/sq ft = Rs. 180,000

**Agar user area NAHI bataye aur rate puchay:**
✅ SAHI: "Deemak ka rate Rs. 80 per sq ft hai."
❌ GALAT: "Apna area bataen to main total cost bata sakta hoon."
❌ GALAT: "Lekin per sq ft rate yeh hai."

**🚨 ABSOLUTE BAN - KABHI YEH NA KARO:**
- ❌ NEVER say "Agar aap..."
- ❌ NEVER say "Apna area bataen..."
- ❌ NEVER say "Main aapko bata sakta hoon agar..."
- ❌ NEVER ask questions back

**SAHI JAWAB KI EXAMPLES:**
Q: "Tusi or keri keri service provide karday o?"
A: "Hum yeh services provide karte hain: Termite Treatment, Termite Proofing, General Pest Control, Bed Bug Treatment, Fumigation, Rodent Control, aur Cockroach Control."

Q: "Deemak ka ilaj kaise hota hai?"
A: "Hum property inspect karte hain, foundation ke around holes drill karte hain, soil mein termiticide inject karte hain, aur tamam wooden structures ka treatment karte hain. 5 saal ki guarantee hai."

Q: "Kya treatment ke dauran ghar khali karna hoga?"
A: "Fumigation ke liye haan - 24-48 ghante ke liye khali karna hoga. Termite treatment ke liye aap reh sakte hain lekin treated areas se 4-6 ghante door rahein."

**GENDER RULES (MASCULINE/FEMININE):**
- Use MASCULINE for: deemak, khatmal, service, treatment, guarantee
  ✅ "Deemak KA rate" (NOT "ki rate")
  ✅ "Service KI guarantee" (NOT "ka guarantee")
  ✅ "Treatment KI cost" (NOT "ka cost")

Jawab (SEEDHA, NO EXTRA INFO):"""

# ============================================================================

URDU_PROMPT = """آپ Eastern Services pest control bot ہیں۔

Context: {context}

Question: {question}

⚠️ ضروری اصول - خلاف ورزی = ناکامی:
1. صرف اردو میں جواب دیں
2. صرف Context کی معلومات استعمال کریں - کبھی سوال واپس نہ پوچھیں
3. صرف وہی جواب دیں جو صارف نے پوچھا - اضافی معلومات نہ دیں
4. جواب 2-4 جملوں میں
5. سلام نہیں جب تک صارف سلام نہ کرے
6. **رابطہ معلومات:** فون: +92 336 1101234، ای میل: easternservices.pk@gmail.com (ہمیشہ یہ استعمال کریں!)
7. **گارنٹی مدت:** دیمک کا علاج = 5 سال، دیمک پروفنگ = 10 سال، عام کیڑوں کا کنٹرول = 1 ماہ، کھٹمل کا علاج = 3 ماہ

**🚨 ریٹ/قیمت کے اصول - بالکل فالو کریں:**
- ❌ ریٹ/قیمت کبھی نہ بتائیں جب تک سوال میں یہ نہ ہو: ریٹ، قیمت، لاگت، چارجز، کتنا، کتنی
- ❌ رقبہ کبھی نہ پوچھیں جب تک صارف نے کیڑے کا مسئلہ + لاگت کا حساب دونوں نہ بتایا ہو
- ✅ اگر "کیا سروسز ہیں؟" پوچھا، تو صرف سروسز بتائیں - ریٹ نہیں
- ✅ اگر طریقہ/عمل پوچھا، تو صرف عمل بتائیں - ریٹ نہیں
- ✅ اگر گارنٹی پوچھی، تو صرف گارنٹی کی مدت بتائیں - ریٹ نہیں

**حساب کے اصول (صرف اگر صارف لاگت/ریٹ پوچھے):**
- اگر صارف رقبہ بتائے (مرلہ، کنال، مربع فٹ)، تو کل لاگت کا حساب لگائیں
- 1 مرلہ = 225 مربع فٹ، 1 کنال = 20 مرلہ = 4,500 مربع فٹ
- فارمولا: رقبہ (مربع فٹ) × ریٹ فی مربع فٹ = کل لاگت
- مثال: 10 مرلہ = 2,250 مربع فٹ × Rs. 80/مربع فٹ = Rs. 180,000

**اگر صارف رقبہ نہیں بتائے اور ریٹ پوچھے:**
✅ صحیح: "دیمک کا ریٹ Rs. 80 فی مربع فٹ ہے۔"
❌ غلط: "براہ کرم اپنا رقبہ بتائیں تاکہ میں کل لاگت بتا سکوں۔"
❌ غلط: "لیکن فی مربع فٹ ریٹ یہ ہے۔"

**🚨 مطلق پابندی - کبھی یہ نہ کریں:**
- ❌ کبھی نہ کہیں "اگر آپ..."
- ❌ کبھی نہ کہیں "اپنا رقبہ بتائیں..."
- ❌ کبھی نہ کہیں "میں آپ کو بتا سکتا ہوں اگر..."
- ❌ کبھی سوال واپس نہ پوچھیں

**صحیح جوابات کی مثالیں:**
Q: "آپ کون کون سی سروسز فراہم کرتے ہیں؟"
A: "ہم یہ سروسز فراہم کرتے ہیں: دیمک کا علاج، دیمک پروفنگ، عام کیڑوں کا کنٹرول، کھٹمل کا علاج، فیومیگیشن، چوہوں کا کنٹرول، اور کاکروچ کنٹرول۔"

Q: "دیمک کے علاج کا طریقہ کیا ہے؟"
A: "ہم پراپرٹی کا معائنہ کرتے ہیں، بنیاد کے ارد گرد سوراخ کرتے ہیں، مٹی میں ٹرمیٹیسائیڈ انجیکٹ کرتے ہیں، اور تمام لکڑی کی ساختوں کا علاج کرتے ہیں۔ 5 سال کی گارنٹی ہے۔"

Q: "کیا علاج کے دوران گھر خالی کرنا ہوگا؟"
A: "فیومیگیشن کے لیے ہاں - 24-48 گھنٹے کے لیے خالی کرنا ہوگا۔ دیمک کے علاج کے لیے آپ رہ سکتے ہیں لیکن علاج شدہ علاقوں سے 4-6 گھنٹے دور رہیں۔"

**صنف کے اصول (مذکر/مؤنث):**
- مذکر استعمال کریں: دیمک، کھٹمل، سروس، علاج، گارنٹی
  ✅ "دیمک کا ریٹ" (نہ کہ "کی ریٹ")
  ✅ "سروس کی گارنٹی" (نہ کہ "کا گارنٹی")
  ✅ "علاج کی لاگت" (نہ کہ "کا لاگت")

جواب (سیدھا، کوئی اضافی معلومات نہیں):"""

# ============================================================================
# CREATE RAG CHAIN FUNCTION
# ============================================================================

def create_rag_chain(language='english'):
    """Create RAG chain with language-specific prompt"""

    if language == 'urdu':
        prompt_template = URDU_PROMPT
    elif language == 'roman_urdu':
        prompt_template = ROMAN_URDU_PROMPT
    else:
        prompt_template = ENGLISH_PROMPT

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    chain = (
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

print("✅ ULTIMATE Language-specific prompts created!")
print("   🚨 NEW RULE: Answer ONLY what is asked - NO rate obsession!")
print("="*80)



✅ ULTIMATE Language-specific prompts created!
   🚨 NEW RULE: Answer ONLY what is asked - NO rate obsession!


## Cell 10 - Fallback Mechanism

In [13]:
# Cell 10: ULTIMATE Fallback Mechanism (URDU KEYWORD BOOSTING)

FALLBACK_RESPONSES = {
    'english': """I apologize, but I don't have specific information about that in my current knowledge base.

However, I can help you with:
- Termite control and treatment
- Bed bug elimination
- Cockroach, rodent, and mosquito control
- Service pricing and packages
- Booking and scheduling

For immediate assistance, please contact Eastern Services:
📞 Phone: +92 336 1101234
📧 Email: easternservices.pk@gmail.com

Would you like to know about any of these services?""",

    'roman_urdu': """Maafi chahta hoon, lekin mere paas is baray mein specific information nahi hai.

Main aapki in cheezon mein madad kar sakta hoon:
- Termite (deemak) control aur treatment
- Bed bugs (khatmal) ka khatma
- Cockroach, rats, aur mosquito control
- Service rates aur packages
- Booking aur scheduling

Foran madad ke liye Eastern Services se contact karein:
📞 Phone: +92 336 1101234
📧 Email: easternservices.pk@gmail.com

Kya aap in services ke baray mein jaanna chahenge?""",

    'urdu': """معذرت، لیکن میرے پاس اس بارے میں مخصوص معلومات نہیں ہیں۔

میں آپ کی ان چیزوں میں مدد کر سکتا ہوں:
- دیمک کنٹرول اور علاج
- کھٹمل کا خاتمہ
- کاکروچ، چوہے، اور مچھر کنٹرول
- سروس کی قیمتیں اور پیکجز
- بکنگ اور شیڈولنگ

فوری مدد کے لیے Eastern Services سے رابطہ کریں:
📞 فون: +92 336 1101234
📧 ای میل: easternservices.pk@gmail.com

کیا آپ ان خدمات کے بارے میں جاننا چاہیں گے؟"""
}

def check_retrieval_quality(docs, question):
    """Check if retrieved documents are relevant (URDU KEYWORD BOOSTING)."""
    if not docs or len(docs) == 0:
        return False, 0.0

    question_lower = question.lower()

    # Extract question words
    question_words = set(question_lower.split())

    # Common stop words (expanded for multilingual)
    common_words = {
        'the', 'a', 'an', 'is', 'are', 'what', 'how', 'when', 'where',
        'kya', 'hai', 'ka', 'ki', 'ke', 'mein', 'se', 'ko', 'ne', 'par',
        'tay', 'da', 'di', 'nu', 'te', 'vich', 'wich', 'ah', 'oye',
        'my', 'your', 'his', 'her', 'their', 'our', 'this', 'that',
        'mery', 'meray', 'tera', 'tuhadaha', 'tussin', 'ma', 'tusi',
        'will', 'ga', 'gi', 'gay', 'hoga', 'hogi', 'hogay', 'ho'
    }

    # Important pest control keywords (multilingual + transliterated Urdu)
    important_keywords = {
        # Pests
        'deemak', 'termite', 'bed', 'bug', 'khatmal', 'cockroach',
        'mosquito', 'rat', 'rodent', 'ant', 'chiti', 'machar', 'chooha', 'chuhay',
        # Services
        'treatment', 'treatement', 'control', 'spray', 'fumigation',
        'inspection', 'service', 'ilaj', 'ilaaj',
        # Pricing
        'rate', 'price', 'cost', 'fee', 'qeemat', 'kimat', 'rupay', 'rupees',
        'marla', 'kanal', 'sq', 'square', 'per',
        # Time/Warranty
        'guarantee', 'warranty', 'time', 'waqt', 'kitni', 'kitna',
        # Areas
        'ghar', 'house', 'home', 'area', 'construction', 'wood', 'kaar', 'lakri',
        # Transliterated Urdu (from transliterate_urdu_to_roman function)
        'meray', 'mery', 'residence', 'white', 'ant', 'bedbug', 'mattress',
        'roach', 'kitchen', 'pest', 'dengue', 'malaria', 'mouse', 'sugar',
        'assurance', 'much', 'many', 'timber', 'building', 'tameer'
    }

    # Remove common words
    question_words = question_words - common_words

    if len(question_words) == 0:
        return True, 1.0

    # Count keyword matches
    important_matches = sum(1 for word in question_words if word in important_keywords)

    # Calculate relevance
    total_matches = 0
    for doc in docs:
        # Handle both dict and object formats
        if isinstance(doc, dict):
            doc_text = doc['page_content'].lower()
        else:
            doc_text = doc.page_content.lower()

        # Count regular word matches
        matches = sum(1 for word in question_words if word in doc_text)
        total_matches += matches

    # Calculate base relevance score
    if len(question_words) > 0 and len(docs) > 0:
        relevance_score = total_matches / (len(question_words) * len(docs))
    else:
        relevance_score = 0.0

    # BOOST score if important keywords are present
    if important_matches > 0:
        keyword_boost = important_matches * 0.25  # INCREASED from 0.20 to 0.25
        relevance_score = min(1.0, relevance_score + keyword_boost)

    # SPECIAL HANDLING FOR URDU: Lower threshold + Extra boost
    # Detect if query has Urdu script
    import re
    urdu_pattern = r'[\u0600-\u06FF]'
    has_urdu_script = re.search(urdu_pattern, question)

    if has_urdu_script:
        # For Urdu queries, add EXTRA BOOST (0.15) to compensate for transliteration loss
        relevance_score = min(1.0, relevance_score + 0.15)
        # Use VERY LOW threshold (0.08 instead of 0.15)
        threshold = 0.08
    else:
        # For Roman/English queries, use standard threshold
        threshold = 0.15

    return relevance_score >= threshold, relevance_score

print("✅ Fallback mechanism loaded!")
print("   Improved multilingual relevance checking enabled!")
print("   ⭐ URDU BOOST: +0.15 bonus + 0.08 threshold (vs. 0.15 for others)")
print("="*80)



✅ Fallback mechanism loaded!
   Improved multilingual relevance checking enabled!
   ⭐ URDU BOOST: +0.15 bonus + 0.08 threshold (vs. 0.15 for others)


## Cell 11 - Conversation Memory (Redis + Supabase)

In [14]:
# Cell 11: Conversation Memory with Redis and Supabase

import uuid
import time

# Generate unique session ID for this Colab session
SESSION_ID = str(uuid.uuid4())[:8]

def save_to_redis(question, answer, language):
    """Save conversation to Redis (fast, temporary cache)."""
    if redis_client is None:
        return

    try:
        # Store in Redis with 1-hour expiry
        key = f"chat:{SESSION_ID}:history"
        conversation_entry = {
            'question': question,
            'answer': answer,
            'language': language,
            'timestamp': time.time()
        }

        # Add to list (keep last 10 exchanges)
        redis_client.lpush(key, json.dumps(conversation_entry))
        redis_client.ltrim(key, 0, 9)  # Keep only last 10
        redis_client.expire(key, 3600)  # 1 hour expiry

    except Exception as e:
        print(f"⚠️  Redis save failed: {e}")

def save_to_supabase(question, answer, language):
    """Save conversation to Supabase (permanent storage)."""
    if SessionLocal is None:
        return

    try:
        db = SessionLocal()
        conversation = ConversationHistory(
            session_id=SESSION_ID,
            user_question=question,
            bot_response=answer,
            language=language,
            meta_data=json.dumps({'source': 'colab'})  # Changed from metadata to meta_data
        )
        db.add(conversation)
        db.commit()
        db.close()
    except Exception as e:
        print(f"⚠️  Supabase save failed: {e}")

def get_conversation_context():
    """Get recent conversation history from Redis."""
    if redis_client is None:
        return "No previous conversation."

    try:
        key = f"chat:{SESSION_ID}:history"
        history = redis_client.lrange(key, 0, 2)  # Last 3 exchanges

        if not history:
            return "No previous conversation."

        context = "Previous conversation:\n"
        for entry_json in reversed(history):
            entry = json.loads(entry_json)
            context += f"\nUser: {entry['question'][:100]}\n"
            context += f"Assistant: {entry['answer'][:150]}...\n"

        return context
    except Exception as e:
        return "No previous conversation."

def clear_conversation():
    """Clear conversation history."""
    if redis_client:
        try:
            key = f"chat:{SESSION_ID}:history"
            redis_client.delete(key)
            print("✅ Conversation history cleared!")
        except Exception as e:
            print(f"⚠️  Clear failed: {e}")

print(f"✅ Conversation memory initialized!")
print(f"   Session ID: {SESSION_ID}")
print("="*80)

✅ Conversation memory initialized!
   Session ID: f6673e71


## Cell 12 - Evaluation Metrics

In [15]:
# Cell 12: Evaluation Metrics

evaluation_log = []

def evaluate_response(question, answer, retrieved_docs, response_time, language):
    """Evaluate response quality."""
    metrics = {
        'timestamp': datetime.now().isoformat(),
        'question': question[:100],
        'answer_length': len(answer),
        'response_time': response_time,
        'language': language,
        'num_docs_retrieved': len(retrieved_docs),
        'relevance_score': 0.0,
        'completeness_score': 0.0,
        'overall_score': 0.0
    }

    # Relevance score
    question_words = set(question.lower().split())
    answer_words = set(answer.lower().split())
    common_words = {'the', 'a', 'an', 'is', 'are', 'what', 'how', 'kya', 'hai', 'ka'}
    question_words = question_words - common_words

    if len(question_words) > 0:
        overlap = len(question_words.intersection(answer_words))
        metrics['relevance_score'] = min(1.0, overlap / len(question_words))

    # Completeness score
    if 50 <= len(answer) <= 500:
        metrics['completeness_score'] = 1.0
    elif len(answer) < 50:
        metrics['completeness_score'] = len(answer) / 50
    else:
        metrics['completeness_score'] = max(0.5, 500 / len(answer))

    # Overall score
    metrics['overall_score'] = (
        metrics['relevance_score'] * 0.6 +
        metrics['completeness_score'] * 0.4
    )

    evaluation_log.append(metrics)
    return metrics

def display_metrics(metrics, show_details=True):
    """Display evaluation metrics."""
    if show_details:
        print("\n" + "="*60)
        print("RESPONSE EVALUATION")
        print("="*60)
        print(f"Language: {metrics['language']}")
        print(f"Response Time: {metrics['response_time']:.2f}s")
        print(f"Answer Length: {metrics['answer_length']} characters")
        print(f"\nScores:")
        print(f"  Relevance: {metrics['relevance_score']:.2f}/1.00")
        print(f"  Completeness: {metrics['completeness_score']:.2f}/1.00")
        print(f"  Overall: {metrics['overall_score']:.2f}/1.00")

        if metrics['overall_score'] >= 0.8:
            print(f"  Quality: ✅ Excellent")
        elif metrics['overall_score'] >= 0.6:
            print(f"  Quality: ✓ Good")
        elif metrics['overall_score'] >= 0.4:
            print(f"  Quality: ⚠️  Fair")
        else:
            print(f"  Quality: ❌ Poor")
        print("="*60)

def get_evaluation_summary():
    """Get evaluation summary."""
    if not evaluation_log:
        print("No evaluations yet!")
        return

    print("\n" + "="*60)
    print("EVALUATION SUMMARY")
    print("="*60)
    print(f"Total Queries: {len(evaluation_log)}")

    avg_relevance = sum(e['relevance_score'] for e in evaluation_log) / len(evaluation_log)
    avg_completeness = sum(e['completeness_score'] for e in evaluation_log) / len(evaluation_log)
    avg_overall = sum(e['overall_score'] for e in evaluation_log) / len(evaluation_log)
    avg_response_time = sum(e['response_time'] for e in evaluation_log) / len(evaluation_log)

    print(f"\nAverage Scores:")
    print(f"  Relevance: {avg_relevance:.2f}/1.00")
    print(f"  Completeness: {avg_completeness:.2f}/1.00")
    print(f"  Overall: {avg_overall:.2f}/1.00")
    print(f"\nAverage Response Time: {avg_response_time:.2f}s")

    languages = {}
    for e in evaluation_log:
        lang = e['language']
        if lang not in languages:
            languages[lang] = 0
        languages[lang] += 1

    print(f"\nQueries by Language:")
    for lang, count in languages.items():
        print(f"  {lang}: {count}")

    print("="*60)

print("✅ Evaluation metrics loaded!")
print("="*80)

✅ Evaluation metrics loaded!


## Cell 13 - Complete Chatbot Function

In [18]:
# Cell 13: Complete Chatbot Function (CLEAN - NO DEBUG OUTPUT)

from IPython.display import display, Markdown
import time

def show_welcome_message():
    """Display welcome message when chatbot initializes."""
    welcome_msg = """
# Assalam-o-Alaikum! 👋

**Welcome to Eastern Services Pest Control Chatbot**

Main aapki kya madad kar sakta hoon? (How may I help you?)

---

**Available Services:**
- Termite Treatment & Proofing
- Bed Bug Treatment
- General Pest Control
- Fumigation Services
- Rodent Control
- Cockroach Control

**Contact Us:**
📞 Phone: +92 336 1101234
📧 Email: easternservices.pk@gmail.com

---
"""
    display(Markdown(welcome_msg))

def ask_chatbot(question):
    """
    Complete chatbot function - CLEAN VERSION (NO DEBUG OUTPUT)

    Features:
    - PGVector retrieval (cloud-based)
    - Fallback mechanism
    - Conversation memory (Redis + Supabase) - FOR LOGGING ONLY
    - Multilingual support

    Returns only the response text - no debug info, no metrics, no scores.
    """
    start_time = time.time()

    # Detect language and greeting
    language = detect_language(question)
    greeting_type = detect_greeting(question)

    try:
        # Retrieve relevant documents from PGVector
        docs = retrieve_similar_chunks(question, k=3)

        # Check retrieval quality
        is_relevant, score = check_retrieval_quality(docs, question)

        # If retrieval quality is poor, use fallback
        if not is_relevant:
            fallback_msg = FALLBACK_RESPONSES.get(language, FALLBACK_RESPONSES['english'])

            # Add greeting if user greeted
            if greeting_type:
                if greeting_type == 'urdu_greeting':
                    fallback_msg = "وعليكم السلام ورحمة الله وبركاته\n\n" + fallback_msg
                elif greeting_type == 'roman_urdu_greeting':
                    fallback_msg = "Waalaikum Assalam wa Rahmatullahi wa Barakatuh\n\n" + fallback_msg
                elif greeting_type == 'english_greeting':
                    fallback_msg = "Hello! " + fallback_msg

            # Display response (CLEAN - NO DEBUG)
            display(Markdown(fallback_msg))

            # Save to memory (FOR LOGGING ONLY)
            save_to_redis(question, fallback_msg, language)
            save_to_supabase(question, fallback_msg, language)

            return

        # Create chain and get response
        chain = create_rag_chain(language)

        # Format docs for chain
        context = "\n\n".join([doc['page_content'] if isinstance(doc, dict) else doc.page_content for doc in docs])

        # Get response from LLM
        response = chain.invoke({"context": context, "question": question})
        response = clean_response(response, language, question)

        # Add greeting if user greeted
        if greeting_type:
            if greeting_type == 'urdu_greeting':
                response = "وعليكم السلام ورحمة الله وبركاته\n\n" + response
            elif greeting_type == 'roman_urdu_greeting':
                response = "Waalaikum Assalam wa Rahmatullahi wa Barakatuh\n\n" + response
            elif greeting_type == 'english_greeting':
                response = "Hello! " + response

        # Display response (CLEAN - NO DEBUG, NO METRICS, NO SCORES)
        display(Markdown(response))

        # Save to memory (FOR LOGGING ONLY - NOT FOR LLM CONTEXT)
        save_to_redis(question, response, language)
        save_to_supabase(question, response, language)

    except Exception as e:
        error_msg = "Maafi chahta hoon, kuch technical issue ho raha hai. Kya aap phir se try kar sakte hain?"
        if language == 'urdu':
            error_msg = "معذرت، کچھ تکنیکی مسئلہ ہو رہا ہے۔ کیا آپ پھر سے کوشش کر سکتے ہیں؟"
        elif language == 'english':
            error_msg = "Sorry, there's a technical issue. Please try again."

        display(Markdown(error_msg))
        print(f"⚠️ Error: {str(e)[:100]}")  # Only print error to console, not to user

# Show welcome message when this cell runs
show_welcome_message()

print("✅ Chatbot ready! Ask your question using: ask_chatbot('your question')")
print("="*80)




# Assalam-o-Alaikum! 👋

**Welcome to Eastern Services Pest Control Chatbot**

Main aapki kya madad kar sakta hoon? (How may I help you?)

---

**Available Services:**
- Termite Treatment & Proofing
- Bed Bug Treatment
- General Pest Control
- Fumigation Services
- Rodent Control
- Cockroach Control

**Contact Us:**
📞 Phone: +92 336 1101234  
📧 Email: easternservices.pk@gmail.com

---


✅ Chatbot ready! Ask your question using: ask_chatbot('your question')


## Cell 14 - Testing

In [20]:
# Cell 14: Test the Complete Cloud-Based System

print("\n" + "="*80)
print("TESTING COMPLETE CLOUD-BASED RAG SYSTEM")
print("="*80)
print("Data Source: GitHub Repository")
print("Embeddings: Supabase PGVector + Cohere")
print("="*80)

# Test various scenarios
ask_chatbot("Mery ghar mein deemak hai,mera 5 marlay ka ghar hai ,to us ka rate kia ho ga?")
# ask_chatbot("Guarantee kitni hai?", show_debug=True, show_metrics=True)
# ask_chatbot("What is the bed bug treatment process?", show_debug=True, show_metrics=True)
# ask_chatbot("Tusi or keri keri service provide karday o?", show_debug=True, show_metrics=True)
# Test 5: PURE URDU (Termite Rate)
# ask_chatbot("میرے گھر میں دیمک ہے، ریٹ کیا ہو گا؟", show_debug=True, show_metrics=True)

# Test 6: PURE URDU (Guarantee)
# ask_chatbot("گارنٹی کتنی ہے؟", show_debug=True, show_metrics=True)
# ask_chatbot("دیمک کے علاج کا طریقہ کیا ہے؟", show_debug=True, show_metrics=True)
# ask_chatbot("فیومیگیشن کیسے ہوتا ہے؟", show_debug=True, show_metrics=True)
# ask_chatbot("کیا آج ہی سروس مل سکتی ہے؟", show_debug=True, show_metrics=True)
# ask_chatbot("کیا کوئی ڈسکاؤنٹ ہے؟", show_debug=True, show_metrics=True)
ask_chatbot("کیا علاج کے دوران گھر خالی کرنا ہوگا؟")

# Show evaluation summary
get_evaluation_summary()

print("\n✅ Cloud-based system testing complete!")
print("="*80)


TESTING COMPLETE CLOUD-BASED RAG SYSTEM
Data Source: GitHub Repository
Embeddings: Supabase PGVector + Cohere


Deemak ka rate Rs. 80 per sq ft hai. Apka 5 marlay ka ghar hai, lekin humein area sq ft mein maloom hona chahiye. 1 Marla = 225 sq ft, to apka 5 marlay ka ghar = 5 × 225 = 1125 sq ft. Total cost = 1125 sq ft × Rs. 80/sq ft = Rs. 90,000. Humari guarantee period 1 saal hai. Agar aapko aur information chahiye, to humse contact karein: Phone: +92 336 1101234, Email: easternservices.pk@gmail.com.

   🔄 Transliterated: 'کیا علاج کے دوران گھر خالی کرنا ہوگا؟...' → 'kya kia what ilaj treatment control serv...'


فیومیگیشن کے لیے ہاں - 24-48 گھنٹے کے لیے خالی کرنا ہوگا۔ دیمک کے علاج کے لیے آپ رہ سکتے ہیں لیکن علاج شدہ علاقوں سے 4-6 گھنٹے دور رہیں۔


EVALUATION SUMMARY
Total Queries: 1

Average Scores:
  Relevance: 0.46/1.00
  Completeness: 1.00/1.00
  Overall: 0.68/1.00

Average Response Time: 3.26s

Queries by Language:
  roman_urdu: 1

✅ Cloud-based system testing complete!


## Cell 15 - Usage Instructions

In [47]:
# 🎉 System Ready!

## How to Use:

### Basic Query:
ask_chatbot("meray kaar cha wood da kam oye ah ,tay deemak bohat lagi ah wood wich,tay ma oda treatement karwana chana")
ask_chatbot("Eastern services k baray may kuch batay?services kaya kaya provide karta hai?aur CEO kon hai? aur head office kidar hai")
ask_chatbot("Cockroach k baray may batay k wo kis teran k hotay hai?")
ask_chatbot("bed bug k treatement ka kaya process hai?")
ask_chatbot("pest sa hum kesay bach saktay hai ,means koi precautions bata dan.")
ask_chatbot("Termite k treatement ka kaya process hai?")
ask_chatbot("commercial k liya Monthly Plans kaya hai aur payment k baray may b bata dan")
ask_chatbot("Seasonal Pest Calendar kaya hai?")
ask_chatbot("mera 10 marlay ka ghar hai ,meray ghar may mechar bohat ,to ma fumigation karwana chahta hun ,to us ki quotation bana dan.")




### With Debug Info:
ask_chatbot("rate ki ah demaak treatement da", show_debug=True, show_metrics=True)
### Clear Conversation:
clear_conversation()

### View Evaluation Summary:
get_evaluation_summary()


  [🔄 Punjabi detected → Mapped to Roman Urdu]

> QUESTION: meray kaar cha wood da kam oye ah ,tay deemak bohat lagi ah wood wich,tay ma oda treatement karwana ...

--- RAG Response ---


Meray pass deemak ka ilaj karwana chahye, hum Eastern Services mein advanced pre-construction aur post-construction anti-termite treatments provide karte hain. Hum property inspect karte hain, foundation ke around holes drill karte hain, soil mein termiticide inject karte hain, aur tamam wooden structures ka treatment karte hain. 5 saal ki guarantee hai, aur aap humse contact kar sakte hain: Phone: +92 336 1101234, Email: easternservices.pk@gmail.com.

  [🔄 Punjabi detected → Mapped to Roman Urdu]

> QUESTION: Eastern services k baray may kuch batay?services kaya kaya provide karta hai?aur CEO kon hai? aur he...

--- RAG Response ---


Eastern Services yeh services provide karta hai: Termite Control, Fumigation Services, Bed Bug Treatment, Cockroach Control, Rodent Control, Mosquito/Dengue Control, aur Commercial aur Residential Pest Control services. CEO ka naam Raja Zahid Iqbal hai. Head office Islamabad mein hai, Office # 12, 2nd Floor, Siddique Trade Centre, Blue Area. Phone: +92 336 1101234, Email: easternservices.pk@gmail.com.

  [🔄 Punjabi detected → Mapped to Roman Urdu]

> QUESTION: Cockroach k baray may batay k wo kis teran k hotay hai?

--- RAG Response ---


...

  [🔄 Punjabi detected → Mapped to Roman Urdu]

> QUESTION: mera 10 marlay ka ghar hai ,meray ghar may mechar bohat ,to ma fumigation karwana chahta hun ,to us ...

--- RAG Response ---


Aapke 10 marlay ke ghar ke liye Mosquito Fogging ki quotation Rs. 3,500 hai. Agar aapko aur bhi services ki zaroorat hai, to humari services mein shamil hain: Termite Treatment, Termite Proofing, General Pest Control, Bed Bug Treatment, Fumigation, Rodent Control, aur Cockroach Control. Humse contact karne ke liye phone: +92 336 1101234, aur email: easternservices.pk@gmail.com par rabta karein.

  [🔄 Punjabi detected → Mapped to Roman Urdu]

> QUESTION: rate ki ah demaak treatement da
  [Language: ROMAN_URDU]
  [Session ID: a5f050b4]
  [Relevance Score: 0.83]
  [Using Fallback: False]

--- RAG Response ---


Deemak ka rate Rs. 80 per sq ft hai. Hum 1 saal ki guarantee dete hain. Agar aapko aur information chahiye, to humse +92 336 1101234 par phone kar sakte hain ya easternservices.pk@gmail.com par email kar sakte hain.


RESPONSE EVALUATION
Language: roman_urdu
Response Time: 11.39s
Answer Length: 215 characters

Scores:
  Relevance: 0.33/1.00
  Completeness: 1.00/1.00
  Overall: 0.60/1.00
  Quality: ✓ Good
✅ Conversation history cleared!

EVALUATION SUMMARY
Total Queries: 21

Average Scores:
  Relevance: 0.37/1.00
  Completeness: 1.00/1.00
  Overall: 0.62/1.00

Average Response Time: 9.43s

Queries by Language:
  roman_urdu: 11
  english: 3
  urdu: 7


**COMPREHENSIVE TESTING - Context Memory + Accuracy**

In [48]:
# Cell 16: COMPREHENSIVE TESTING - Context Memory + Accuracy

print("\n" + "="*80)
print("COMPREHENSIVE TESTING - MEMORY + ACCURACY")
print("="*80)

# Clear previous conversation
clear_conversation()

print("\n" + "="*80)
print("TEST SET 1: CONTEXT MEMORY (ROMAN URDU)")
print("="*80)

# Sequential conversation to test memory
ask_chatbot("Aap kon kon si services provide karte hain?", show_debug=True)
ask_chatbot("Termite ka rate kya hai?", show_debug=True)  # Should remember context
ask_chatbot("Guarantee kitni hai?", show_debug=True)  # Should know it's about termite

# Clear for next test
clear_conversation()

print("\n" + "="*80)
print("TEST SET 2: CONTEXT MEMORY (PUNJABI)")
print("="*80)

ask_chatbot("Tussin keri keri service provide karday o?", show_debug=True)
ask_chatbot("Deemak da rate ki ah?", show_debug=True)  # Should remember context
ask_chatbot("Guarantee kitni ae?", show_debug=True)  # Should know it's about deemak

# Clear for next test
clear_conversation()

print("\n" + "="*80)
print("TEST SET 3: ACCURACY - FIXED VALUES")
print("="*80)

# Test specific, unchangeable values
ask_chatbot("What is the termite treatment rate?", show_debug=True)  # Should say Rs. 80/sq ft
ask_chatbot("How long is the guarantee period?", show_debug=True)  # Should say 1 year or 10 years
ask_chatbot("What is the contact number?", show_debug=True)  # Should give exact number
ask_chatbot("What is the email address?", show_debug=True)  # Should give exact email
ask_chatbot("Which areas do you serve?", show_debug=True)  # Should list cities

# Clear for next test
clear_conversation()

print("\n" + "="*80)
print("TEST SET 4: PURE URDU (NASTALIQ SCRIPT)")
print("="*80)

ask_chatbot("میرے گھر میں دیمک ہے، ریٹ کیا ہو گا؟", show_debug=True)
ask_chatbot("گارنٹی کتنی ہے؟", show_debug=True)
ask_chatbot("کون کون سی خدمات ہیں؟", show_debug=True)

# Clear for next test
clear_conversation()

print("\n" + "="*80)
print("TEST SET 5: MULTILINGUAL MIX (REAL SCENARIO)")
print("="*80)

ask_chatbot("What services do you offer?", show_debug=True)  # English
ask_chatbot("Deemak ka rate kya hai?", show_debug=True)  # Switch to Roman Urdu
ask_chatbot("Mera 10 marlay ka ghar hai", show_debug=True)  # Switch to Roman Urdu
ask_chatbot("Guarantee kitni ae?", show_debug=True)  # Punjabi
ask_chatbot("کون سے علاقے؟", show_debug=True)  # Pure Urdu

# Show final summary
get_evaluation_summary()

print("\n✅ Comprehensive testing complete!")
print("="*80)


COMPREHENSIVE TESTING - MEMORY + ACCURACY
✅ Conversation history cleared!

TEST SET 1: CONTEXT MEMORY (ROMAN URDU)

> QUESTION: Aap kon kon si services provide karte hain?
  [Language: ROMAN_URDU]
  [Session ID: a5f050b4]
  [Relevance Score: 0.24]
  [Using Fallback: False]

--- RAG Response ---


Hum yeh services provide karte hain: Office Pest Control Package, Restaurant and Food Service Package, Hotel and Hospitality Package, Warehouse and Storage Package, Termite Treatment, Termite Proofing, General Pest Control, Bed Bug Treatment, Fumigation, Rodent Control, aur Cockroach Control. Humara contact number hai +92 336 1101234 aur email hai easternservices.pk@gmail.com.


> QUESTION: Termite ka rate kya hai?
  [Language: ROMAN_URDU]
  [Session ID: a5f050b4]
  [Relevance Score: 1.00]
  [Using Fallback: False]

--- RAG Response ---


Termite ka rate Rs. 80 per sq ft hai. Yeh rate post-construction termite treatment ke liye hai, jo 1 saal ki guarantee ke saath aata hai. Pre-construction termite proofing ka rate Rs. 65 per sq ft hai, jo 10 saal ki guarantee ke saath aata hai.


> QUESTION: Guarantee kitni hai?
  [Language: ROMAN_URDU]
  [Session ID: a5f050b4]
  [Relevance Score: 0.83]
  [Using Fallback: False]

--- RAG Response ---


...

Traceback (most recent call last):
  File "/tmp/ipython-input-695585012.py", line 83, in ask_chatbot
    response = chain.invoke({"context": context, "question": question})
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3129, in invoke
    input_ = context.run(step.invoke, input_, config)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 398, in invoke
    self.generate_prompt(
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 1117, in generate_prompt
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 927, i

  [Relevance Score: 0.83]
  [Using Fallback: False]
❌ ERROR: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kb71fnxvfhprgqz76g731v8g` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99405, Requested 1882. Please try again in 18m31.967999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

> QUESTION: کون سے علاقے؟
  [Language: URDU]
  [Session ID: a5f050b4]
   🔄 Transliterated: 'کون سے علاقے؟...' → 'kon kaun which who what se say from with...'


Traceback (most recent call last):
  File "/tmp/ipython-input-695585012.py", line 83, in ask_chatbot
    response = chain.invoke({"context": context, "question": question})
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3129, in invoke
    input_ = context.run(step.invoke, input_, config)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 398, in invoke
    self.generate_prompt(
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 1117, in generate_prompt
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 927, i

  [Relevance Score: 0.15]
  [Using Fallback: False]
❌ ERROR: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kb71fnxvfhprgqz76g731v8g` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99404, Requested 2927. Please try again in 33m33.984s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

EVALUATION SUMMARY
Total Queries: 35

Average Scores:
  Relevance: 0.37/1.00
  Completeness: 1.00/1.00
  Overall: 0.62/1.00

Average Response Time: 9.38s

Queries by Language:
  roman_urdu: 17
  english: 8
  urdu: 10

✅ Comprehensive testing complete!


Traceback (most recent call last):
  File "/tmp/ipython-input-695585012.py", line 83, in ask_chatbot
    response = chain.invoke({"context": context, "question": question})
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/base.py", line 3129, in invoke
    input_ = context.run(step.invoke, input_, config)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 398, in invoke
    self.generate_prompt(
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 1117, in generate_prompt
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py", line 927, i